In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("white")
%matplotlib inline

In [2]:
donations_cohort_with_cycle=pd.read_pickle('donations_cohort_with_cycle.pkl')

In [3]:
donations_cohort_with_cycle.groupby('cohort_gift_pattern').sum()

,lapsed_count,recovered_count,max_consec_giving_years,total_giving_years,count_2013,count_2014,count_2015,count_2016,count_2017,count_yr0,...,amount_05_2017,amount_06_2017,amount_07_2017,amount_08_2017,amount_09_2017,amount_10_2017,amount_11_2017,amount_12_2017,amount_total,count_total
cohort_gift_pattern,,,,,,,,,,,,,,,,,,,,,
"0,",3246751,1698013,1696680,1698013,281367,299265,343664,459121,568128,2160468,...,1513719.92,1346215.27,2430744.12,4660638.35,3614630.25,3381341.86,2653097.34,2732646.05,1.108747e+08,1951545
"0,1,",456484,324471,324008,324471,64778,102879,90901,133044,189491,351927,...,669986.44,470229.87,681269.53,1379156.40,1285395.33,1154510.77,1301913.91,1650338.26,3.663071e+07,581093
"0,1,2,",140611,113307,112917,113307,26461,47280,63866,83705,113547,111393,...,406391.15,286160.91,440258.05,652367.16,609036.92,504306.19,595653.51,820301.07,2.145994e+07,334859
"0,1,2,3,",64826,55642,55124,55642,18753,42483,56392,62173,59272,56260,...,256549.75,169966.88,258912.92,507154.02,413960.30,429247.78,547307.06,651570.56,1.679730e+07,239073
"0,1,2,3,4,",75395,70626,70626,70626,106916,174536,150126,143233,153440,127899,...,540893.94,429600.39,628210.06,1041223.65,976890.30,977064.73,1138716.31,1550895.28,4.428904e+07,728251
"0,1,2,4,",17175,12431,8901,12431,10974,12197,8673,1470,6534,12653,...,26429.85,17497.27,20652.61,47481.46,51619.98,39111.52,63445.55,109491.95,2.361383e+06,39848
"0,1,3,",39132,24601,16268,24601,6848,13560,10706,8209,6029,17694,...,16821.11,15506.57,26961.28,52175.80,48118.31,30767.99,32472.96,69326.56,2.995362e+06,45352
"0,1,3,4,",16037,11830,6200,11830,5221,8010,2292,5243,10504,7792,...,36626.51,28702.58,34569.66,57856.73,68852.78,51097.72,65494.68,135615.08,2.091935e+06,31270
"0,1,4,",14173,9353,5988,9353,6417,6798,1454,1,4562,8262,...,15205.59,9166.41,13938.24,34097.56,29615.97,22492.52,34206.41,44463.84,1.064790e+06,19232


### 1. Re-creating Pete's csv

In [21]:
grouped=df.groupby('first_gift_year').sum().reset_index()

In [23]:
# here is the sql query I ran 

# select
# first_gift_year,

#sum(lifecycle_cohort_0_first_gift) as first_gift_cnt,
first_gift_cnt=grouped.cohort_0_first_gift

# sum(lifecycle_cohort_1_second_year)   as second_year_cnt,
second_year_cnt=grouped.cohort_1_second_year

# sum(lifecycle_cohort_2_lapsed_after_second)   as lapsed_after_second_cnt,
lapsed_after_second_cnt=grouped.cohort_2_lapsed_after_second

# sum(lifecycle_cohort_2_second_with_gap)   as second_with_gap_cnt,
second_with_gap_cnt=grouped.cohort_2_second_with_gap

# sum(lifecycle_cohort_2_lapsed_after_first_long)   as lapsed_after_first_long_cnt
lapsed_after_first_long_cnt=grouped.cohort_2_lapsed_after_first_long

# sum(lifecycle_cohort_2_multi_year)   as multi_year_cnt,
multi_year_cnt=grouped.cohort_2_multi_year


# ((sum(lifecycle_cohort_1_second_year * 1.0) ) / 
#  (sum(lifecycle_cohort_0_first_gift  * 1.0))) * 100.0 as second_year,
second_year=second_year_cnt/first_gift_cnt*100

# ((sum(lifecycle_cohort_1_second_year * cohort_amount_in_year_in_1) ) / 
#  (sum(lifecycle_cohort_1_second_year  * 1.0))) / 100.0 as second_year_avg,
second_year_avg=second_year_avg=(donations_cohort_with_cycle.groupby('first_gift_year').sum()['amount_yr1'].reset_index()['amount_yr1'])/second_year_cnt

# ((sum(lifecycle_cohort_1_lapsed_after_first * 1.0) ) / 
#  (sum(lifecycle_cohort_0_first_gift  * 1.0))) * 100.0 as lapsed_after_first,
lapsed_after_first=grouped.cohort_1_lapsed_after_first/first_gift_cnt*100

# ((sum(lifecycle_cohort_2_multi_year * 1.0) ) / 
#  (sum(lifecycle_cohort_1_second_year  * 1.0))) * 100.0 as multi_year,
multi_year=grouped.cohort_2_multi_year/second_year_cnt*100


# ((sum(lifecycle_cohort_2_multi_year * cohort_amount_in_year_in_2) ) / 
#  (sum(lifecycle_cohort_2_multi_year  * 1.0))) / 100.0 as multi_year_avg,
multi_year_avg=(donations_cohort_with_cycle.query('cohort_2_multi_year > 0').groupby('first_gift_year').sum()['amount_yr2'].reset_index()['amount_yr2'])/multi_year_cnt


# ((sum(lifecycle_cohort_2_lapsed_after_second * 1.0) ) / 
#  (sum(lifecycle_cohort_1_second_year  * 1.0))) * 100.0 as lapsed_after_second,
lapsed_after_second=lapsed_after_second_cnt/second_year_cnt*100

# ((sum(lifecycle_cohort_2_second_with_gap * 1.0) ) / 
#  (sum(lifecycle_cohort_1_lapsed_after_first  * 1.0))) * 100.0 as second_with_gap,
second_with_gap=second_with_gap_cnt/grouped.cohort_1_lapsed_after_first*100


# ((sum(lifecycle_cohort_2_second_with_gap * cohort_amount_in_year_in_2) ) / 
#  (sum(lifecycle_cohort_2_second_with_gap  * 1.0))) / 100.0 as second_with_gap_avg,
second_with_gap_avg=(donations_cohort_with_cycle.query('cohort_2_second_with_gap > 0').groupby('first_gift_year').sum()['amount_yr2'].reset_index()['amount_yr2'])/second_with_gap_cnt

# ((sum(lifecycle_cohort_2_lapsed_after_first_long * 1.0) ) / 
#  (sum(lifecycle_cohort_1_lapsed_after_first  * 1.0))) * 100.0 as lapsed_after_first_long,
lapsed_after_first_long=lapsed_after_first_long_cnt/grouped.cohort_1_lapsed_after_first*100

# from donationslifecycles
# group by first_gift_year

# which gives the attached csv which should show some of the power we can get in sql

In [28]:
data = {
    'cohort_year':grouped.first_gift_year,
    'first_gift_cnt':first_gift_cnt,
    'second_year':second_year,
'second_year_cnt':second_year_cnt,
'second_year_avg':second_year_avg,
'lapsed_after_first':lapsed_after_first,
'multi_year':multi_year,
'multi_year_cnt':multi_year_cnt,
'multi_year_avg':multi_year_avg,
'lapsed_after_second':lapsed_after_second,
'lapsed_after_second_cnt':lapsed_after_second_cnt,
'second_with_gap':second_with_gap,
'second_with_gap_cnt':second_with_gap_cnt,
'second_with_gap_avg':second_with_gap_avg,
'lapsed_after_first_long':lapsed_after_first_long,
'lapsed_after_first_long_cnt':lapsed_after_first_long_cnt
}

In [30]:
df4 = pd.DataFrame(data) 

In [31]:
df4

,cohort_year,first_gift_cnt,second_year,second_year_cnt,second_year_avg,lapsed_after_first,multi_year,multi_year_cnt,multi_year_avg,lapsed_after_second,lapsed_after_second_cnt,second_with_gap,second_with_gap_cnt,second_with_gap_avg,lapsed_after_first_long,lapsed_after_first_long_cnt
0,2012-01-01,141,23.404255,33,497.610303,76.595745,36.363636,12,1597.260833,63.636364,21,1.851852,2,32.140000,98.148148,106
1,2013-01-01,320069,19.411439,62130,245.738033,80.588561,41.976501,26080,407.968347,58.023499,36050,6.262333,16153,120.565399,93.737667,241786
2,2014-01-01,320673,15.176519,48667,159.161182,84.823481,32.340189,15739,299.721817,67.659811,32928,5.803548,15786,108.981826,94.196452,256220
3,2015-01-01,355137,12.974711,46078,179.707233,87.025289,33.254482,15323,366.020322,66.745518,30755,5.345258,16520,111.114785,94.654742,292539
4,2016-01-01,445598,12.688118,56538,185.287068,87.311882,18.279741,10335,205.318955,81.718490,46202,2.143114,8338,91.576951,97.856629,380721
5,2017-01-01,434991,6.894396,29990,151.714345,93.105604,0.000000,0,NaN,0.000000,0,0.000000,0,NaN,0.000000,0
6,2018-01-01,147945,0.000000,0,NaN,0.000000,NaN,0,NaN,NaN,0,NaN,0,NaN,NaN,0


In [71]:
#df4.to_csv('df4.csv')

In [32]:
df5=df4

In [34]:
#rename columns to make more intuitive
df5.columns=['cohort_year', 
             'no_donors_yr0', 
             'retention_rate_yr1', 
             'no_donors_yr1', 
             'avg_donation_yr1', 
             'lapse_rate_yr1', 
             'retention_rate_from_previous_yr_yr2', 
             'no_donors_012_y2', 
             'avg_donation_012_donors_yr2', 
             'lapse_rate_from_previous_year_yr2', 
             'no_donors_lapsed_from_previous_year_yr2', 
             'percent_donors_returning_yr2_after_lapsing_yr1_02', 
             'no_donors_02_yr2', 
             'avg_donation_of_02_donor', 
             'percent_donors_lapsing_yr2_after_lapsing_yr1', 
             'no_donors_lapsing_yr2_after_lapsing_yr1']

In [36]:
#make lapsed columns negative values
df5['lapse_rate_yr1']=df5['lapse_rate_yr1']*-1

In [42]:
#add column of no_donors_lapsed_yr1 and make negative
df5['no_donors_lapsed_yr1']=(df5['no_donors_yr0']-df5['no_donors_yr1'])*-1

In [48]:
#make lapsed columns negative values
df5['no_donors_lapsed_from_previous_year_yr2']=df5['no_donors_lapsed_from_previous_year_yr2']*-1

In [49]:
#make lapsed columns negative values
df5['no_donors_lapsing_yr2_after_lapsing_yr1']=df5['no_donors_lapsing_yr2_after_lapsing_yr1']*-1

In [85]:
df5['retention_rate_from_previous_yr_yr2']=df5['retention_rate_from_previous_yr_yr2']*-1
df5['lapse_rate_from_previous_year_yr2'] =df5['lapse_rate_from_previous_year_yr2']*-1
df5['percent_donors_returning_yr2_after_lapsing_yr1_02']=df5['percent_donors_returning_yr2_after_lapsing_yr1_02']*-1
df5['percent_donors_lapsing_yr2_after_lapsing_yr1']=df5['percent_donors_lapsing_yr2_after_lapsing_yr1']*-1

In [84]:
df5

,cohort_year,no_donors_yr0,retention_rate_yr1,no_donors_yr1,avg_donation_yr1,lapse_rate_yr1,retention_rate_from_previous_yr_yr2,no_donors_012_y2,avg_donation_012_donors_yr2,lapse_rate_from_previous_year_yr2,no_donors_lapsed_from_previous_year_yr2,percent_donors_returning_yr2_after_lapsing_yr1_02,no_donors_02_yr2,avg_donation_of_02_donor,percent_donors_lapsing_yr2_after_lapsing_yr1,no_donors_lapsing_yr2_after_lapsing_yr1,no_donors_lapsed_yr1
0,2012-01-01,141,23.404255,33,497.610303,-76.595745,36.363636,12,1597.260833,63.636364,-21,1.851852,2,32.140000,98.148148,-106,-108
1,2013-01-01,320069,19.411439,62130,245.738033,-80.588561,41.976501,26080,407.968347,58.023499,-36050,6.262333,16153,120.565399,93.737667,-241786,-257939
2,2014-01-01,320673,15.176519,48667,159.161182,-84.823481,32.340189,15739,299.721817,67.659811,-32928,5.803548,15786,108.981826,94.196452,-256220,-272006
3,2015-01-01,355137,12.974711,46078,179.707233,-87.025289,33.254482,15323,366.020322,66.745518,-30755,5.345258,16520,111.114785,94.654742,-292539,-309059
4,2016-01-01,445598,12.688118,56538,185.287068,-87.311882,18.279741,10335,205.318955,81.718490,-46202,2.143114,8338,91.576951,97.856629,-380721,-389060
5,2017-01-01,434991,6.894396,29990,151.714345,-93.105604,0.000000,0,NaN,0.000000,0,0.000000,0,NaN,0.000000,0,-405001
6,2018-01-01,147945,0.000000,0,NaN,-0.000000,NaN,0,NaN,NaN,0,NaN,0,NaN,NaN,0,-147945


In [81]:
fig = px.bar(df5, x="cohort_year", y='no_donors_yr0',
            title="Year 0: Number of Donors",
            color_discrete_sequence =['seagreen'])
 #barmode='group', labels={'index':'year', 'y':'amount ($)'}, 
fig.update_xaxes(type='category')
fig.show()

In [82]:
fig = px.bar(df5, x="cohort_year", y=['no_donors_yr1', 'no_donors_lapsed_yr1'],
            title="Year 1: Retention vs Lapse (# Donors)",
            color_discrete_sequence =['seagreen','red'])
 #barmode='group', labels={'index':'year', 'y':'amount ($)'}, 
fig.update_xaxes(type='category')
fig.show()

In [83]:
fig = px.bar(df5, x="cohort_year", y=['retention_rate_yr1', 'lapse_rate_yr1'],
            title="Year 1: Retention vs Lapse (% of Donors)",
            color_discrete_sequence =['seagreen','red'])
 #barmode='group', labels={'index':'year', 'y':'amount ($)'}, 
fig.update_xaxes(type='category')
fig.show()

In [80]:
fig = px.bar(df5, x="cohort_year", y=['no_donors_012_y2', 'no_donors_02_yr2', 'no_donors_lapsed_from_previous_year_yr2', 'no_donors_lapsing_yr2_after_lapsing_yr1'],
            title="Year 2: Make up of donors Retention vs Lapse (# of Donors)", 
             color_discrete_sequence =['seagreen', 'mediumseagreen','orangered','red'])
 #barmode='group', labels={'index':'year', 'y':'amount ($)'}, 
fig.update_xaxes(type='category')
#fig.update_traces(marker_color='green','red')
fig.show()

In [87]:
fig = px.bar(df5, x="cohort_year", y=['retention_rate_from_previous_yr_yr2', 'lapse_rate_from_previous_year_yr2', 'percent_donors_returning_yr2_after_lapsing_yr1_02', 'percent_donors_lapsing_yr2_after_lapsing_yr1'],
            title="Year 2: Make up of donors Retention vs Lapse (% of Donors)", 
             color_discrete_sequence =['seagreen', 'mediumseagreen','orangered','red'])
 #barmode='group', labels={'index':'year', 'y':'amount ($)'}, 
fig.update_xaxes(type='category')
fig.show()